# Productividad VTR

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta  # Asegúrate de importar timedelta

In [2]:
import pandas as pd

# Ruta del archivo
ruta = r"C:\Users\wduran\Downloads\soportepostventa_Attendants_Status_2026-02-06_2026-02-11.xlsx"

# Leer el archivo Excel (por defecto lee la primera hoja)
prod = pd.read_excel(ruta)

In [3]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
prod['MAIL'] = prod['AgentIdentity'].str.split('@').str[0]

In [4]:
# Lista de columnas a convertir
cols_to_int = [
    "Online", "Invisible", "Pause", "Offline",
    "OpenTicketOnline", "OpenTicketInvisible", "WithoutTicketOnline"
]

# Reemplazar nulos por 0 y convertir a entero
for col in cols_to_int:
    prod[col] = prod[col].fillna(0).astype(int)


In [5]:
# Convertir el campo a datetime
prod["DateTimeRef"] = pd.to_datetime(prod["DateTimeRef"], dayfirst=True, errors="coerce")

# Crear campo Fecha en formato dd-mm-aaaa
prod["Fecha"] = prod["DateTimeRef"].dt.strftime("%d-%m-%Y")

# Crear campo INTERVALO con hora en formato HH:MM
prod["INTERVALO"] = prod["DateTimeRef"].dt.strftime("%H:%M")


In [6]:
prod.sample(1)

,AgentIdentity,BotId,Teams,Device,DateTimeRef,Online,Invisible,Pause,Offline,OpenTicketOnline,OpenTicketInvisible,WithoutTicketOnline,WithoutTicketInvisible,MAIL,Fecha,INTERVALO
11296,julieth.espinosa@e.vtr.cl,soportepostventa@msging.net,[Abuse],Web,2026-02-10 16:00:00,59,0,0,0,59,0,0,0.0,julieth.espinosa,10-02-2026,16:00


In [7]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha.csv"
blip_ha = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'

# Ajustar las opciones para mostrar todas las columnas
pd.set_option('display.max_columns', None)

C:\Users\wduran\AppData\Local\Temp\ipykernel_24684\3964556920.py:3: DtypeWarning: Columns (17,20,21,22,23,24,25,26,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,73,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,119,120,121,122) have mixed types. Specify dtype option on import or set low_memory=False.
  blip_ha = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'


In [8]:
blip_ha = blip_ha.sort_values(by="Team", ascending=True)

In [9]:
# Eliminar registros cuyo campo Team contenga la palabra "videollamada" 
blip_ha = blip_ha[~blip_ha["Team"].str.contains("videollamada", case=False, na=False)]

In [10]:
blip_ha

,SequentialId,CustomerIdentity,AgentIdentity,Status,StorageDate,ExpirationDate,CloseDate,Team,Closed,Tags,QueueTime,FirstResponseTime,AverageResponseTime,CustomerName,CustomerEmail,CustomerGender,CustomerCity,CustomerPhoneNumber,AgentName,AgentEmail,campaignOriginator,botOrigin,campaignMessageTemplate,campaignId,Menu Hogar,priorityTag,Campaign,Menu Principal,team,rut,activeMessageFileName,actividad,activa,headline,body,source_type,source_id,source_url,media_id,tiporedproductotoa,NumeroSerie,SourceId,SourceUrl,SourceTitle,SourceType,direccion,nombreplanprincipal,svatotal,databeneficiosincluidos,preciopermanente,dctomensual,montomensualplan,totalproductoadicional,oct-13,variable1,nombre,mes,montopago,numboleta,fechapago,fechacorte,cargomensual,montomensual,urlarr[1],Fecha,HORA,INTERVALO,NEUTRA,Semana,Def_IBR,Llave,Flag_Outbound,MAIL,Rama,Defle_meta,fecha,access_token,activity_id,appt_number,number,img,meta_ad_id,meta_ad_headline,meta_ad_type,meta_ad_url,medios,rut_cliente,imagen,monto,numero,medio,Template,menuPrincipal,ticketCerrado,name,sept-13,marca,montofinpegado,montofin,url,product,productprecio,plan,cargo,desc,cargofijo,sva,valor,servicio,subflowId,number6b1b088dfa84473bbd7d857d6ac847b3,glosa,teste1,code,activecampaign:6a08f94abb504971aea43c27e7714d7e,medio06daee1056af4e2fba0f6389ecf30adc,activecampaign:fd3b0d1994dc426da152bae6b20b5907,activecampaign:be235cd8482f43a490f3f810e68d8f90,EMPRESA_PAIS,motivo,problema,afectacion,adicional
1073817,3801471,94f17c1a-a35a-4370-b955-a4980d29d7a8@tunnel.ms...,julian.merchan%40e.vtr.cl@blip.ai,ClosedAttendant,2026-02-11 09:54:46,NaN,2026-02-11 10:05:15 -03:00,abuse,True,"[""DX de servicio Principal""]",00:10:23,NaN,0,Hilda Gomez,NaN,NaN,NaN,56956950235,JULIAN ANDRES MERCHAN BELEÃO,julian.merchan@e.vtr.cl,NaN,"Peru Oriente 057 Edif. P4 Depto 11, Los Andes",Servicio Hogar,5968306-3,NaN,NaN,Consulta Saldo,NaN,NaN,5968306-3,https://i.imgur.com/XhrREcP.jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026-02-11,09:54:46,09:00,0,2,0,56956950235_2026-02-11,0,julian.merchan,NaN,0.0,NaN,"Peru Oriente 057 Edif. P4 Depto 11, Los Andes",26127794.0,1-3HDXJOYH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATENTO COLOMBIA,NaN,NaN,wimt_visita_impecable_vtr,c0450e90-07c0-4a03-9d04-0c8b4b48fbd1
818739,3446240,8e419eaa-564b-4297-8070-a9b69110f79f@tunnel.ms...,julian.bueno%40e.vtr.cl@blip.ai,ClosedAttendant,2025-11-26 12:26:35,NaN,2025-11-26 12:28:30 -03:00,abuse,True,"[""Visita TÃ©cnica reag/desi/incum""]",00:00:15,00:00:34,0,Morelia,NaN,NaN,NaN,56967176290,Julian David Bueno Largo,julian.bueno@e.vtr.cl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6906239-3,25531245.0,"Eloy Caviedes 500, Valparaiso",NaN,1-3G8DTUDR,wY0PCIZ7wXM,NaN,Servicio Hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-26,12:26:35,12:00,0,4,0,56967176290_2025-11-26,0,julian.bueno,NaN,0.0,Excepciones,854c63d5-4210-45c1-9f8b-92bf28907c99,NaN,6906239-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATENTO COLOMBIA,NaN,NaN,NaN,NaN
768742,3282359,5677e2eb-16ad-447a-ab86-e01a8c94a5da@tunnel.ms...,lina.raigoza%40e.vtr.cl@blip.ai,ClosedAttendant,2025-10-09 22:08:13,NaN,2025-10-15 23:14:30 -03:00,abuse,True,"[""abuse""]",10:57:50,NaN,0,TeteUC????,NaN,NaN,NaN,56965697109,Lina Raigoza,lina.raigoza@e.vtr.cl,77767676-7,Inactividad,NaN,NaN,NaN,Excepciones,NaN,NaN,NaN,77767676-7,reset,reset,NaN,reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-10-09,22:08:13,22:00,0,1,0,56965697109_2025-10-09,0,lina.raigoza,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [11]:
# Define the file path
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Dotaciones\VTR\Dotacion_CLAROVTR_20260130.xlsx"
# Read the specific sheet named 'Dotacion' from the Excel file
dotacion = pd.read_excel(archivo, sheet_name='Dotacion')  # Specify the sheet name

In [12]:
dotacion_filtrada = dotacion[
    (dotacion['ESTADO_CONTRATO'].str.lower() == 'vigente') &
    (dotacion['PLATAFORMA'].str.lower().isin(['POSTVENTA NO VOZ FIJO']))
]


In [13]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
dotacion_filtrada['MAIL'] = dotacion_filtrada['CORREO_ELECTRONICO'].str.split('@').str[0]

In [14]:
# Creamos versiones en minúscula del campo MAIL para el join
prod['MAIL'] = prod['MAIL'].str.lower()
dotacion_filtrada['MAIL'] = dotacion_filtrada['MAIL'].str.lower()
blip_ha['MAIL'] = blip_ha['MAIL'].str.lower()

In [15]:
# Hacer merge directo por MAIL
prod = prod.merge(
    dotacion_filtrada[['MAIL', 'EMPRESA_PAIS']], 
    on='MAIL', 
    how='left'
)

# Crear campo MARCA: 'VTR' si coincide, vacío si no
prod['MARCA'] = prod['EMPRESA_PAIS'].notna().map({True: 'VTR', False: ''})

In [16]:
prod

,AgentIdentity,BotId,Teams,Device,DateTimeRef,Online,Invisible,Pause,Offline,OpenTicketOnline,OpenTicketInvisible,WithoutTicketOnline,WithoutTicketInvisible,MAIL,Fecha,INTERVALO,EMPRESA_PAIS,MARCA
0,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 00:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,06-02-2026,00:00,NaN,
1,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 01:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,06-02-2026,01:00,NaN,
2,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 02:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,06-02-2026,02:00,NaN,
3,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 03:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,06-02-2026,03:00,NaN,
4,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 04:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,06-02-2026,04:00,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20707,zoila.sanchez@e.vtr.cl,soportepostventa@msging.net,"[Reagendamiento, ]",Web,2026-02-11 19:00:00,0,0,0,59,0,0,0,0.0,zoila.sanchez,11-02-2026,19:00,NaN,
20708,zoila.sanchez@e.vtr.cl,soportepostventa@msging.net,"[Reagendamiento, ]",Web,2026-02-11 20:00:00,0,0,0,59,0,0,0,0.0,zoila.sanchez,11-02-2026,20:00,NaN,
20709,zoila.sanchez@e.vtr.cl,soportepostventa@msging.net,"[Reagendamiento, ]",Web,2026-02-11 21:00:00,0,0,0,59,0,0,0,0.0,zoila.sanchez,11-02-2026,21:00,NaN,
20710,zoila.sanchez@e.vtr.cl,soportepostventa@msging.net,"[Reagendamiento, ]",Web,2026-02-11 22:00:00,0,0,0,59,0,0,0,0.0,zoila.sanchez,11-02-2026,22:00,NaN,


In [17]:
# Asegurarnos de que DateTimeRef es datetime
prod["DateTimeRef"] = pd.to_datetime(prod["DateTimeRef"], dayfirst=True, errors="coerce")

# Reconvertir campo Fecha a formato YYYY-MM-DD
prod["Fecha"] = prod["DateTimeRef"].dt.strftime("%Y-%m-%d")

In [18]:
# --- 3) Contar SequentialId por MAIL-Fecha-INTERVALO en blip_ha ---
blip_counts = (
    blip_ha
    .groupby(["MAIL", "Fecha", "INTERVALO"], as_index=False)["SequentialId"]
    .count()
    .rename(columns={"SequentialId": "Atenciones"})
)

# --- 4) Merge a prod para traer Atenciones (dejar 0 si no hay match) ---
prod = prod.merge(
    blip_counts,
    on=["MAIL", "Fecha", "INTERVALO"],
    how="left"
)

prod["Atenciones"] = prod["Atenciones"].fillna(0).astype(int)

In [19]:
# Opcional si Online pudiese venir como texto
prod['H-Productiva'] = (prod['Online'] >= 10).astype(int)

In [20]:
prod

,AgentIdentity,BotId,Teams,Device,DateTimeRef,Online,Invisible,Pause,Offline,OpenTicketOnline,OpenTicketInvisible,WithoutTicketOnline,WithoutTicketInvisible,MAIL,Fecha,INTERVALO,EMPRESA_PAIS,MARCA,Atenciones,H-Productiva
0,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 00:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,2026-02-06,00:00,NaN,,0,0
1,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 01:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,2026-02-06,01:00,NaN,,0,0
2,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 02:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,2026-02-06,02:00,NaN,,0,0
3,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 03:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,2026-02-06,03:00,NaN,,0,0
4,adriana.chanchari@e.vtr.cl,soportepostventa@msging.net,"[Abuse, Contacto_telefonico]",Web,2026-02-06 04:00:00,0,0,0,59,0,0,0,0.0,adriana.chanchari,2026-02-06,04:00,NaN,,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20707,zoila.sanchez@e.vtr.cl,soportepostventa@msging.net,"[Reagendamiento, ]",Web,2026-02-11 19:00:00,0,0,0,59,0,0,0,0.0,zoila.sanchez,2026-02-11,19:00,NaN,,0,0
20708,zoila.sanchez@e.vtr.cl,soportepostventa@msging.net,"[Reagendamiento, ]",Web,2026-02-11 20:00:00,0,0,0,59,0,0,0,0.0,zoila.sanchez,2026-02-11,20:00,NaN,,0,0
20709,zoila.sanchez@e.vtr.cl,soportepostventa@msging.net,"[Reagendamiento, ]",Web,2026-02-11 21:00:00,0,0,0,59,0,0,0,0.0,zoila.sanchez,2026-02-11,21:00,NaN,,0,0
20710,zoila.sanchez@e.vtr.cl,soportepostventa@msging.net,"[Reagendamiento, ]",Web,2026-02-11 22:00:00,0,0,0,59,0,0,0,0.0,zoila.sanchez,2026-02-11,22:00,NaN,,0,0


In [21]:
# Eliminar filas con NaN en EMPRESA_PAIS
prod = prod[prod['EMPRESA_PAIS'].notna()].copy()

In [22]:
prod

,AgentIdentity,BotId,Teams,Device,DateTimeRef,Online,Invisible,Pause,Offline,OpenTicketOnline,OpenTicketInvisible,WithoutTicketOnline,WithoutTicketInvisible,MAIL,Fecha,INTERVALO,EMPRESA_PAIS,MARCA,Atenciones,H-Productiva


In [23]:
# Asegúrate de que el campo Fecha esté en formato datetime
prod["Fecha"] = pd.to_datetime(prod["Fecha"], errors="coerce")

# Crear campo Año
prod["Año"] = prod["Fecha"].dt.year

# Crear campo Mes (número del mes)
prod["Mes"] = prod["Fecha"].dt.month

In [24]:
prod.sample(1)

ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
# Ahora sí, guardás el archivo
prod.to_excel(r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Productividad\Productividad_OCT.xlsx", index=False)

# Productividad CLARO

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta  # Asegúrate de importar timedelta

In [ ]:
import pandas as pd
 
# Ruta del archivo
ruta = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Productividad\Copia de soporteposventahogarclara_Attendants_Status_2025-07-01_2025-07-31.xlsx"

# Leer el archivo Excel (por defecto lee la primera hoja)
prodc = pd.read_excel(ruta)

In [ ]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
prodc['MAIL'] = prodc['AgentIdentity'].str.split('@').str[0]

In [ ]:
# Lista de columnas a convertir
cols_to_int = [
    "Online", "Invisible", "Pause", "Offline",
    "OpenTicketOnline", "OpenTicketInvisible", "WithoutTicketOnline"
]

# Reemplazar nulos por 0 y convertir a entero
for col in cols_to_int:
    prodc[col] = prodc[col].fillna(0).astype(int)


In [ ]:
# Convertir el campo a datetime
prodc["DateTimeRef"] = pd.to_datetime(prodc["DateTimeRef"], dayfirst=True, errors="coerce")

# Crear campo Fecha en formato dd-mm-aaaa
prodc["Fecha"] = prodc["DateTimeRef"].dt.strftime("%d-%m-%Y")

# Crear campo INTERVALO con hora en formato HH:MM
prodc["INTERVALO"] = prodc["DateTimeRef"].dt.strftime("%H:%M")


In [ ]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha.csv"
blip_ha = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'

# Ajustar las opciones para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [ ]:
# Define the file path
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Dotaciones\CLARO\Dotación_CLARO_20250130.xlsx"
# Read the specific sheet named 'Dotacion' from the Excel file
dotacionc = pd.read_excel(archivo, sheet_name='Dotacion')  # Specify the sheet name

In [ ]:
dotacionc.sample(1)

In [ ]:
dotacion_filtradac = dotacionc[
    (dotacionc['ESTADO_CONTRATO'].str.lower() == 'vigente') &
    (dotacionc['PLATAFORMA'].str.lower().isin(['postventa no voz'])) &
    (dotacionc['CARGO'].str.lower() == 'asesor') &
    (dotacionc['ESTADO_ASESOR'].str.lower().isin(['activo']))
]

In [ ]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
dotacion_filtradac['MAIL'] = dotacion_filtradac['CORREO_ELECTRONICO'].str.split('@').str[0]

In [ ]:
# Creamos versiones en minúscula del campo MAIL para el join
prodc['MAIL'] = prodc['MAIL'].str.lower()
dotacion_filtradac['MAIL'] = dotacion_filtradac['MAIL'].str.lower()
blip_ha['MAIL'] = blip_ha['MAIL'].str.lower()

In [ ]:
# Hacer merge directo por MAIL
prodc = prodc.merge(
    dotacion_filtradac[['MAIL', 'EMPRESA_PAIS']], 
    on='MAIL', 
    how='left'
)

# Crear campo MARCA: 'VTR' si coincide, vacío si no
prodc['MARCA'] = prodc['EMPRESA_PAIS'].notna().map({True: 'CLARO', False: ''})

In [ ]:
# Asegurarnos de que DateTimeRef es datetime
prodc["DateTimeRef"] = pd.to_datetime(prodc["DateTimeRef"], dayfirst=True, errors="coerce")

# Reconvertir campo Fecha a formato YYYY-MM-DD
prodc["Fecha"] = prodc["DateTimeRef"].dt.strftime("%Y-%m-%d")

In [ ]:
# --- 3) Contar SequentialId por MAIL-Fecha-INTERVALO en blip_ha ---
blip_counts = (
    blip_ha
    .groupby(["MAIL", "Fecha", "INTERVALO"], as_index=False)["SequentialId"]
    .count()
    .rename(columns={"SequentialId": "Atenciones"})
)

# --- 4) Merge a prod para traer Atenciones (dejar 0 si no hay match) ---
prodc = prodc.merge(
    blip_counts,
    on=["MAIL", "Fecha", "INTERVALO"],
    how="left"
)

prodc["Atenciones"] = prodc["Atenciones"].fillna(0).astype(int)

In [ ]:
# Opcional si Online pudiese venir como texto
prodc['H-Productiva'] = (prodc['Online'] >= 10).astype(int)

In [ ]:
# Eliminar filas con NaN en EMPRESA_PAIS
prodc = prodc[prodc['EMPRESA_PAIS'].notna()].copy()

In [ ]:
prodc

In [ ]:
# Ahora sí, guardás el archivo
prodc.to_excel(r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Productividad\Productividad_CLARO_SEP.xlsx", index=False)